In [ ]:
# Configura framework
from typing import Any
from sincpro_framework import UseFramework, Feature, DataTransferObject, ApplicationService, Middleware

framework = UseFramework("payment-system")

# Dependencies
class Database:
    """Database class to handle transactions"""
    def save_transaction(self, data):
        return f"saved_{id(data)}"
    
    def get_transaction(self, tx_id):
        return {"id": tx_id, "status": "completed"}

class VisaAdapter:
    """Adapter for Visa payment processing"""
    def make_request(self, card_data):
        return {"approved": True, "tx_id": "visa_123"}
    
    def check_status(self, tx_id):
        return "approved"

# Middleware
class ValidatePayment(Middleware):
    """Middleware to validate payment data"""
    
    def __call__(self, dto: Any) -> Any:
        """Validate payment and raise error if invalid"""
        if hasattr(dto, 'amount') and dto.amount <= 0:
            raise ValueError("Amount must be positive")
        if hasattr(dto, 'user_id') and not dto.user_id.strip():
            raise ValueError("User ID is required")
        return dto
    

database = Database()
visa_adapter = VisaAdapter()

framework.add_dependency("database", database)
framework.add_dependency("visa_adapter", visa_adapter)
framework.add_middleware(ValidatePayment())


class Feat(Feature):
    database: Database
    visa_adapter: VisaAdapter

class Services(ApplicationService):
    database: Database
    visa_adapter: VisaAdapter



In [ ]:
class PaymentCommand(DataTransferObject):
    """PaymentCommand"""
    card_number: str
    amount: float
    merchant_id: str

class PaymentResponse(DataTransferObject):
    """PaymentResponse"""
    transaction_id: str
    status: str
    amount: float

class RefundCommand(DataTransferObject):
    """RefundCommand"""

    transaction_id: str
    amount: float
    reason: str

class RefundResponse(DataTransferObject):
    """RefundResponse"""
    refund_id: str
    status: str
    amount: float


class ValidateCommand(DataTransferObject):
    """ValidateCommand"""
    card_number: str
    cvv: str

class ValidateResponse(DataTransferObject):
    """ValidateResponse"""
    is_valid: bool
    card_type: str

In [ ]:
@framework.feature(PaymentCommand)
class PaymentFeature(Feat):
    """Feature to handle payment processing"""
    def execute(self, dto: PaymentCommand) -> PaymentResponse:
        """Execute payment processing
        Args:
            dto (PaymentCommand): Data transfer object containing payment details
        Returns:
            PaymentResponse: Response containing transaction details
        """
        visa_result = self.visa_adapter.make_request({"card": dto.card_number})
        tx_id = self.database.save_transaction(dto)
        return PaymentResponse(
            transaction_id=tx_id,
            status="success",
            amount=dto.amount
        )



@framework.feature(RefundCommand)
class RefundFeature(Feat):
    """Feature to handle refund processing"""

    def execute(self, dto: RefundCommand) -> RefundResponse:
        """Execute refund processing
        Args:
            dto (RefundCommand): Data transfer object containing refund details
        Returns:
            RefundResponse: Response containing refund details
        """
         # Assuming we have a method to get the original transaction
         # and a method to process the refund through the Visa adapter
        original_tx = self.database.get_transaction(dto.transaction_id)
        refund_result = self.visa_adapter.make_request({"refund": dto.amount})
        refund_id = self.database.save_transaction(dto)
        return RefundResponse(
            refund_id=refund_id,
            status="refunded",
            amount=dto.amount
        )


@framework.feature(ValidateCommand)
class ValidateFeature(Feat):
    """Feature to validate payment card details"""
    def execute(self, dto: ValidateCommand) -> ValidateResponse:
        """Execute card validation
        Args:
            dto (ValidateCommand): Data transfer object containing card details
        Returns:
            ValidateResponse: Response indicating if the card is valid
        """
         # Assuming we have a method to check the card status through the Visa adapter
         # and return whether it is approved or not
        validation = self.visa_adapter.check_status(dto.card_number)
        return ValidateResponse(
            is_valid=validation == "approved",
            card_type="visa"
        )




In [ ]:
class MakeTransactionCommand(DataTransferObject):
    """MakeTransactionCommand"""
    card_number: str
    amount: float
    merchant_id: str

class MakeTransactionResponse(DataTransferObject):
    """MakeTransactionResponse"""
    transaction_id: str
    status: str
    amount: float

In [ ]:
@framework.app_service(MakeTransactionCommand)
class MakeTransactionService(Services):
    """Service to handle making a transaction"""
    def execute(self, dto: MakeTransactionCommand) -> MakeTransactionResponse | None:
        """Execute making a transaction
        Args:
            dto (MakeTransactionCommand): Data transfer object containing transaction details
        Returns:
            MakeTransactionResponse: Response containing transaction details
        """
        comand = PaymentCommand(
            card_number=dto.card_number,
            amount=dto.amount,
            merchant_id=dto.merchant_id
        )
        response = self.feature_bus.execute(comand, PaymentResponse)

        validate_cmd = ValidateCommand(card_number=dto.card_number, cvv="123")
        validation = self.feature_bus.execute(validate_cmd, ValidateResponse)
        if validation.is_valid:
            return MakeTransactionResponse(
                transaction_id=response.transaction_id,
                status=response.status,
                amount=response.amount
            )
        
        raise ValueError("Invalid card details")

In [ ]:
response = framework(MakeTransactionCommand(
    card_number="4111111111111111",
    amount=100.0,
    merchant_id="merchant_123"
), MakeTransactionResponse)
response

In [15]:
framework.build_root_bus()

print("=== INSPECCION DEL REGISTRY ===")
print("Feature Registry:")
print(framework.bus.feature_bus.feature_registry)
print("\nApp Service Registry:")
print(framework.bus.app_service_bus.app_service_registry)

print("\n=== FEATURES DOCUMENTADAS ===")
for key, value in framework.bus.feature_bus.feature_registry.items():
    print(f"\n🔧 Key: {key}, Type: {type(key)}")
    print(f"   Value: {value}, Type: {type(value)}")
    if hasattr(value, '__doc__'):
        print(f"   Docstring: {value.__doc__}")

print("\n=== APPLICATION SERVICES DOCUMENTADOS ===")  
for key, value in framework.bus.app_service_bus.app_service_registry.items():
    print(f"\n🎼 Key: {key}, Type: {type(key)}")
    print(f"   Value: {value}, Type: {type(value)}")
    if hasattr(value, '__doc__'):
        print(f"   Docstring: {value.__doc__}")

print("\n=== DEPENDENCIAS ===")
for dep_name, dep_instance in framework.dynamic_dep_registry.items():
    print(f"🔌 {dep_name}: {type(dep_instance).__name__}")
    methods = [m for m in dir(dep_instance) if not m.startswith('_') and callable(getattr(dep_instance, m))]
    print(f"   Methods: {methods}")

2025-07-29T20:23:24.694442Z [debug    ] Framework bus created          app_name=payment-system filename=bus.py func_name=__init__ lineno=137
2025-07-29T20:23:24.694976Z [debug    ] Registered features: {'PaymentCommand', 'ValidateCommand', 'RefundCommand'} app_name=payment-system filename=bus.py func_name=__init__ lineno=138
2025-07-29T20:23:24.695235Z [debug    ] Registered app services: {'MakeTransactionCommand'} app_name=payment-system filename=bus.py func_name=__init__ lineno=141
=== INSPECCION DEL REGISTRY ===
Feature Registry:
{'ValidateCommand': <__main__.ValidateFeature object at 0x7f2baae81040>, 'RefundCommand': <__main__.RefundFeature object at 0x7f2ba98bc410>, 'PaymentCommand': <__main__.PaymentFeature object at 0x7f2ba98bc200>}

App Service Registry:
{'MakeTransactionCommand': <__main__.MakeTransactionService object at 0x7f2baae83380>}

=== FEATURES DOCUMENTADAS ===

🔧 Key: ValidateCommand, Type: <class 'str'>
   Value: <__main__.ValidateFeature object at 0x7f2baae81040>, T

In [ ]:
print("📚 DOCUMENTACIÓN AUTOMÁTICA DEL SISTEMA DE PAGOS")
print("=" * 60)

# DTOs definidos
dto_names = ['PaymentCommand', 'PaymentResponse', 'RefundCommand', 'RefundResponse', 
             'ValidateCommand', 'ValidateResponse', 'MakeTransactionCommand', 'MakeTransactionResponse']

print("\n📦 DTOs (DATOS DE ENTRADA Y SALIDA)")
print("-" * 40)
for dto_name in dto_names:
    if dto_name in globals():
        dto_class = globals()[dto_name]
        print(f"\n🔹 {dto_name}")
        print(f"   Docstring: {dto_class.__doc__}")
        if hasattr(dto_class, '__annotations__'):
            print(f"   Fields: {dict(dto_class.__annotations__)}")

print("\n⚙️ FEATURES (LÓGICA DE NEGOCIO)")
print("-" * 40)
for dto_name, feature_instance in framework.bus.feature_bus.feature_registry.items():
    feature_class = type(feature_instance)
    print(f"\n🔧 {feature_class.__name__}")
    print(f"   Input DTO: {dto_name}")
    print(f"   Docstring: {feature_class.__doc__}")
    
    if hasattr(feature_instance, 'execute'):
        execute_method = getattr(feature_instance, 'execute')
        print(f"   Execute: {execute_method.__doc__}")

print("\n🎼 APPLICATION SERVICES (ORQUESTACIÓN)")
print("-" * 40)
for dto_name, service_instance in framework.bus.app_service_bus.app_service_registry.items():
    service_class = type(service_instance)
    print(f"\n🔹 {service_class.__name__}")
    print(f"   Input DTO: {dto_name}")
    print(f"   Docstring: {service_class.__doc__}")
    
    if hasattr(service_instance, 'execute'):
        execute_method = getattr(service_instance, 'execute')
        print(f"   Execute: {execute_method.__doc__}")

print("\n🔌 DEPENDENCIAS INYECTADAS")
print("-" * 40)
for dep_name, dep_instance in framework.dynamic_dep_registry.items():
    dep_class = type(dep_instance)
    print(f"\n🔸 {dep_name} -> {dep_class.__name__}")
    print(f"   Docstring: {dep_class.__doc__}")
    methods = [m for m in dir(dep_instance) if not m.startswith('_') and callable(getattr(dep_instance, m))]
    print(f"   Methods: {methods}")

print("\n🎯 RESUMEN DEL SISTEMA")
print("-" * 40)
print(f"📊 Features: {len(framework.bus.feature_bus.feature_registry)}")
print(f"📊 Services: {len(framework.bus.app_service_bus.app_service_registry)}")
print(f"📊 Dependencies: {len(framework.dynamic_dep_registry)}")
print(f"📊 DTOs: {len([n for n in dto_names if n in globals()])}")